In [1]:
import os, gc, torch, PIL

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from utils.eval import save_iou_results
from utils.engine import xami_evaluate
from models.load import get_trained_model, get_model_path
from utils.coco_eval import get_eval_params_dict
from data.dataset import ReflacxDataset, OurRadiologsitsDataset
from data.dataset import collate_fn
from utils.transforms import get_transform
from our_radiologist.load import get_anns

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
XAMI_MIMIC_PATH = "D:\XAMI-MIMIC"

use_clinical = False
use_custom_modal = False
use_early_stop_model=True

In [3]:
use_clinical = False
use_custom_modal = False
use_early_stop_model=True

In [4]:
dataset_params_dict = {
    "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
    "with_clinical": use_clinical,
    "using_full_reflacx": False,
    "bbox_to_mask": True,
    "labels_cols": [
        "Enlarged cardiac silhouette",
        "Atelectasis",
        "Pleural abnormality",
        "Consolidation",
        "Pulmonary edema",
        #  'Groundglass opacity', # 6th disease.
    ],
}

detect_eval_dataset = ReflacxDataset(
    **{**dataset_params_dict, "using_full_reflacx": False,},
    transforms=get_transform(train=False),
)

In [5]:
from dataclasses import dataclass

@dataclass
class ModelSetup:
    use_clinical: bool
    use_custom_modal: bool   
    use_early_stop_model: bool
    name: str

all_model_setups = [
    ModelSetup(
        name="original",
        use_clinical= False,
        use_custom_modal=False,
        use_early_stop_model= True,
    ),
        ModelSetup(
        name="custom_without_clinical",
        use_clinical= False,
        use_custom_modal=True,
        use_early_stop_model= True,
    ),
        ModelSetup(
        name="custom_with_clinical",
        use_clinical= True,
        use_custom_modal=True,
        use_early_stop_model= True,
    )
]

In [6]:
disease_idxs = [1, 2, 3, 4, 5]
# disease_idx = 1 #[1, 2, 3, 4, 5]
all_df = {d: {} for d in detect_eval_dataset.labels_cols}


for disease_str in detect_eval_dataset.labels_cols:
    for model_setup in all_model_setups:
        model_path = get_model_path(
            use_custom_modal=model_setup.use_custom_modal,
            use_clinical=model_setup.use_clinical,
            use_early_stop_model=model_setup.use_early_stop_model,
        )

        eval_df = pd.read_csv(
            os.path.join("eval_results", f"{model_path}_{disease_str}.csv"),
            index_col=0,
        )
        all_df[disease_str][model_path] = eval_df



In [7]:
from utils.print import print_title

In [16]:
from collections import OrderedDict

model_dfs = OrderedDict({})

eval_dataset = 'our' # ['test', 'val', 'our']

for model_setup in all_model_setups:
    model_path = get_model_path(
                use_custom_modal=model_setup.use_custom_modal,
                use_clinical=model_setup.use_clinical,
                use_early_stop_model=model_setup.use_early_stop_model,
            )

    # Pick dataset

    model_eval = []

    for disease_str in detect_eval_dataset.labels_cols:
        model_eval.append({
            **dict(all_df[disease_str][model_path][all_df[disease_str][model_path]['dataset'] == eval_dataset].iloc[0]),
            "disease": disease_str, 
        })

    model_dfs[model_setup.name] = pd.DataFrame(model_eval)[['disease', 'AP@[IoU = 0.50:0.95]', 'AR@[IoU = 0.50:0.95]']]

for idx, k in enumerate(model_dfs.keys()):
   if idx == 0:
      # create the merged df 
      merged_df = model_dfs[k].copy()
      merged_df.columns = [ "disease"  if c == "disease" else f"{c}_{k}" for c in merged_df.columns]
   else:
      df = model_dfs[k].copy()
      df.columns = [ "disease"  if c == "disease" else f"{c}_{k}" for c in df.columns]
      merged_df = merged_df.merge(
         df,
         "left", on="disease",
      )


In [17]:
print_title(f"Dataset [{eval_dataset}]")
merged_df

====================Dataset [our]====================


,disease,AP@[IoU = 0.50:0.95]_original,AR@[IoU = 0.50:0.95]_original,AP@[IoU = 0.50:0.95]_custom_without_clinical,AR@[IoU = 0.50:0.95]_custom_without_clinical,AP@[IoU = 0.50:0.95]_custom_with_clinical,AR@[IoU = 0.50:0.95]_custom_with_clinical
0,Enlarged cardiac silhouette,0.023988,0.066667,0.064145,0.112500,0.096686,0.129167
1,Atelectasis,0.023762,0.022222,0.017822,0.033333,0.035644,0.033333
2,Pleural abnormality,0.000000,0.000000,0.005353,0.033333,0.001320,0.022222
3,Consolidation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Pulmonary edema,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
merged_df.to_csv(f"{eval_dataset}_dataset_class_ap.csv")